**Name**: Teo Choun Meng

**UOW ID**: 7919591


# Instructions:

 1. Understand the code and execute the instructions in each cell. If there is no evidence of code execution, NO marks will be awarded.
 2. Answer the following questions using the corresponding Markdown cells.
 3. Submission:
 * Export to PDF: File → Print → Save as PDF (entire notebook).
 * Filename: Lab1_UOWID.pdf
 * Deadline: 26 Oct 2025, 11:55 pm (2 attempts max).



**Q1**. Describe how the input image is handled differently in the DNN versus the CNN in the notebook. Explain why such differences exist. (2 marks)

**Ans**:


**Q2**. Explain what happens in each step of the training loop for CNN vs DNN model. Screen capture the relevant code in the notebook to support your answers(3 marks).

**Ans**:

**Q3**. Comment on the performance of CNN vs DNN.  Relate your observations to the theory covered in Lecture 2 and/or Tutorial 1. (3 marks).

**Ans**:

**Q4**. Report how loss changes across epochs for both CNN and DNN. Relate the loss figures to how well the models are learning. Screen capture the relevant execution results to support your answers.(2 marks).

**Ans**:




# CSCI323 Lab 1: CIFAR-10: CNN vs DNN

This notebook loads CIFAR-10 and trains **two models** on the same data:

- **CNN** (2×Conv→Pool + 3×FC) — similar to the PyTorch tutorial
- **DNN** (3×Fully Connected) — images flattened to vectors

It logs metrics and plots **overlay charts** for training vs validation curves per model.


In [ ]:

#@title (Optional) Install/upgrade libraries
# On standard Colab this is typically not needed.
# !pip install -q torch torchvision torchaudio


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')

## Data: CIFAR-10 loaders (train/test with normalization)

In [ ]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
testloader  = DataLoader(testset,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
len(trainset), len(testset), classes


## Models

In [ ]:

class SmallCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)   # -> 6x28x28
        self.pool  = nn.MaxPool2d(2, 2)   # -> 6x14x14
        self.conv2 = nn.Conv2d(6, 16, 5)  # -> 16x10x10 -> pool -> 16x5x5
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class SimpleDNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Training & Evaluation Utilities

In [ ]:

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc


@torch.no_grad()
def per_class_accuracy(model, loader, classes, device):
    model.eval()
    correct = {c: 0 for c in classes}
    total = {c: 0 for c in classes}

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = outputs.max(1)
        for y, p in zip(labels, pred):
            total[classes[y]] += 1
            if y == p:
                correct[classes[y]] += 1

    return {c: (correct[c] / total[c]) if total[c] > 0 else 0.0 for c in classes}


def plot_curve(values, title, ylabel):
    plt.figure()
    plt.plot(values)
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()


def plot_overlay(train_values, val_values, title, ylabel):
    plt.figure()
    plt.plot(train_values, label='Train')
    plt.plot(val_values, label='Validation')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.show()


## Train Both Models (adjust epochs/hparams below)

In [ ]:

epochs_cnn = 10  # feel free to edit
epochs_dnn = 20  # DNN often needs more epochs
lr_cnn = 0.001
lr_dnn = 0.001

cnn = SmallCNN().to(device)
dnn = SimpleDNN().to(device)

criterion = nn.CrossEntropyLoss()
opt_cnn = optim.Adam(cnn.parameters(), lr=lr_cnn)
opt_dnn = optim.Adam(dnn.parameters(), lr=lr_dnn)

history = {
    'cnn_train_loss': [], 'cnn_train_acc': [],
    'cnn_val_loss': [],   'cnn_val_acc':  [],
    'dnn_train_loss': [], 'dnn_train_acc': [],
    'dnn_val_loss': [],   'dnn_val_acc':  [],
}

print('Training CNN...')
for epoch in range(epochs_cnn):
    tl, ta = train_one_epoch(cnn, trainloader, criterion, opt_cnn, device)
    vl, va = evaluate(cnn, testloader, criterion, device)
    history['cnn_train_loss'].append(tl)
    history['cnn_train_acc'].append(ta)
    history['cnn_val_loss'].append(vl)
    history['cnn_val_acc'].append(va)
    print(f'[CNN] Epoch {epoch+1}/{epochs_cnn}: train_loss={tl:.4f} acc={ta:.3f} | val_loss={vl:.4f} acc={va:.3f}')

print('\nTraining DNN...')
for epoch in range(epochs_dnn):
    tl, ta = train_one_epoch(dnn, trainloader, criterion, opt_dnn, device)
    vl, va = evaluate(dnn, testloader, criterion, device)
    history['dnn_train_loss'].append(tl)
    history['dnn_train_acc'].append(ta)
    history['dnn_val_loss'].append(vl)
    history['dnn_val_acc'].append(va)
    print(f'[DNN] Epoch {epoch+1}/{epochs_dnn}: train_loss={tl:.4f} acc={ta:.3f} | val_loss={vl:.4f} acc={va:.3f}')

print('Done.')


## Overlayed Curves: Training vs Validation (per model)

In [ ]:

# CNN overlays
plot_overlay(history['cnn_train_loss'], history['cnn_val_loss'], 'CNN Loss (Train vs Validation)', 'Loss')
plot_overlay(history['cnn_train_acc'],  history['cnn_val_acc'],  'CNN Accuracy (Train vs Validation)', 'Accuracy')

# DNN overlays
plot_overlay(history['dnn_train_loss'], history['dnn_val_loss'], 'DNN Loss (Train vs Validation)', 'Loss')
plot_overlay(history['dnn_train_acc'],  history['dnn_val_acc'],  'DNN Accuracy (Train vs Validation)', 'Accuracy')


## Per-Class Accuracy (optional)

In [ ]:

cnn_cls = per_class_accuracy(cnn, testloader, classes, device)
dnn_cls = per_class_accuracy(dnn, testloader, classes, device)

print('Per-class accuracy — CNN')
for c in classes:
    print(f'{c:>6s}: {cnn_cls[c]*100:.1f}%')

print('\nPer-class accuracy — DNN')
for c in classes:
    print(f'{c:>6s}: {dnn_cls[c]*100:.1f}%')
